In [1]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *
import joblib


import os,sys
from importlib import reload
import copy
from collections import namedtuple

sys.path.append("../tracker")

In [109]:
import kalmanfilter as KF
import utilities as Util
import datatypes
import vertexfinder as VF

reload(VF)
reload(Util)

<module 'utilities' from '../tracker\\utilities.py'>

In [110]:
tracks=joblib.load("tracks_example.joblib")
# tracks

In [111]:
vf = VF.VertexFitter(debug=True)
vf.parameters["cut_vertex_SeedDist"]=300
vf.parameters["cut_vertex_SeedChi2"]=50
vf.parameters["cut_vertex_TrackAddDist"]=300
vf.parameters["cut_vertex_TrackAddChi2"]=20
vf.parameters["cut_vertex_VertexChi2Reduced"]=5
vertices = vf.run(tracks)
# %timeit -n 2 vf.run(tracks)

VertexSeed(x0=135.14453597458458, y0=9657.452401315408, z0=8795.288278460723, t0=2.354855499088188, cov=0, chi2=31.54249090912529, dist=96.6931207389477, Ntracks=1, trackind1=0, trackind2=1, score=1146.9983605414752)
--- New seed for vertex --- 
  Seed VertexSeed(x0=135.14453597458458, y0=9657.452401315408, z0=8795.288278460723, t0=2.354855499088188, cov=0, chi2=31.54249090912529, dist=96.6931207389477, Ntracks=1, trackind1=0, trackind2=1, score=1146.9983605414752)
 -> Track [2] added to vertex. Vertex chi2_r 0.42; vertex chi2 increment 2.11. Track: (564.7233936034289, 9893.0, 9638.60280683061, 37.166398369151636, 1.1135459601242568, 1, 2.5422267815307698, 0.09727597639302503)
  * Track [3] too far from vertex. Track dist to vertex: 424.04, track chi2 to vertex: 1269.38
Vertex found! track indices: [0, 1, 2] 
 Chi2: 2.1201143608073845; <ValueView x0=172.99302995257312 y0=9540.854244346832 z0=8785.854206965276 t0=-19.452734477525723>


In [112]:
0.01*(8785-7000-1950), 0.01*(-232.8637214423652), 0.01*(9856.048555922185-8547) 

(-1.6500000000000001, -2.3286372144236522, 13.090485559221852)

In [69]:
print(len(vertices))
print(len(vertices[0].tracks))
print(vertices[0])

0


IndexError: list index out of range

In [ ]:
def root_hits_extractor_tracks(Tree, entry):
    Tree.GetEntry(entry)
    c2list = lambda x: np.array([x.at(i) for i in range(x.size())])

    Digi_layer = c2list(Tree.Digi_layer_id)
    # Only select hits in tracking layer
    mask = (Digi_layer>1) & (Digi_layer<=5) 

    Digi_x = c2list(Tree.Digi_x)[mask]
    Digi_y = c2list(Tree.Digi_y)[mask]
    Digi_z = c2list(Tree.Digi_z)[mask]
    Digi_t = c2list(Tree.Digi_time)[mask]
    Digi_track_id = c2list(Tree.Digi_track_id)[mask]
    Digi_layer = Digi_layer[mask]
    Hits = Util.hit.make_hits(Digi_x, Digi_y, Digi_z, Digi_t, Digi_layer)

    track_ids = np.unique(Digi_track_id)
    track_hits  = {}
    for trackid in track_ids:
        mask = Digi_track_id==trackid
        hit_group = []
        for i in np.flatnonzero(mask):
            hit_group.append(Hits[i])
        track_hits[int(trackid)] = hit_group

    return track_hits